In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import unicodedata

In [17]:
# Reading the dataset

traffic = pd.read_csv("original data/trafficlist_forcountry.csv")
forest = pd.read_csv("original data/forest-cover-v1.csv")
air_city = pd.read_csv("original data/aap_air_quality_database_2018_v14.csv", skiprows=2)
air_country = pd.read_csv("original data/【12】GlobalPM25-1998-2022.csv")
weather = pd.read_csv("original data/GlobalWeatherRepository.csv")

In [53]:
weather

,country,location_name,latitude,longitude,timezone,last_updated_epoch,last_updated,temperature_celsius,temperature_fahrenheit,condition_text,...,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,sunrise,sunset,moonrise,moonset,moon_phase,moon_illumination,country_normalized
0,Afghanistan,Kabul,34.52,69.18,Asia/Kabul,1693301400,2023-08-29 14:00,28.8,83.8,Sunny,...,11.1,1,1,05:24 AM,06:24 PM,05:39 PM,02:48 AM,Waxing Gibbous,93,afghanistan
1,Albania,Tirana,41.33,19.82,Europe/Tirane,1693301400,2023-08-29 11:30,27.0,80.6,Partly cloudy,...,29.6,2,3,06:04 AM,07:19 PM,06:50 PM,03:25 AM,Waxing Gibbous,93,albania
2,Algeria,Algiers,36.76,3.05,Africa/Algiers,1693301400,2023-08-29 10:30,28.0,82.4,Partly cloudy,...,7.9,1,1,06:16 AM,07:21 PM,06:46 PM,03:50 AM,Waxing Gibbous,93,algeria
3,Andorra,Andorra La Vella,42.50,1.52,Europe/Andorra,1693301400,2023-08-29 11:30,10.2,50.4,Sunny,...,0.8,1,1,07:16 AM,08:34 PM,08:08 PM,04:38 AM,Waxing Gibbous,93,andorra
4,Angola,Luanda,-8.84,13.23,Africa/Luanda,1693301400,2023-08-29 10:30,25.0,77.0,Partly cloudy,...,203.3,4,10,06:11 AM,06:06 PM,04:43 PM,04:41 AM,Waxing Gibbous,93,angola
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40536,Venezuela,Caracas,10.50,-66.92,America/Caracas,1711641600,2024-03-28 12:00,28.3,82.9,Sunny,...,3.2,1,1,06:27 AM,06:38 PM,09:25 PM,08:29 AM,Waning Gibbous,93,venezuela
40537,Vietnam,Hanoi,21.03,105.85,Asia/Bangkok,1711641600,2024-03-28 23:00,25.0,77.0,Moderate or heavy rain with thunder,...,93.7,4,10,05:54 AM,06:10 PM,08:46 PM,07:26 AM,Waning Gibbous,93,vietnam
40538,Yemen,Sanaa,15.35,44.21,Asia/Aden,1711641600,2024-03-28 19:00,19.6,67.3,Patchy rain nearby,...,11.9,1,1,06:01 AM,06:15 PM,08:53 PM,07:46 AM,Waning Gibbous,93,yemen
40539,Zambia,Lusaka,-15.42,28.28,Africa/Lusaka,1711641600,2024-03-28 18:00,25.0,77.0,Sunny,...,23.0,1,2,06:12 AM,06:11 PM,08:12 PM,08:32 AM,Waning Gibbous,93,zambia


In [18]:
# Data normalization function, to lowercase, remove special chars, and standardize
def normalize_text(text):
    if pd.isna(text):
        return ""
    text = str(text)
    # Convert to lowercase and remove extra whitespace
    text = text.lower().strip()
    # Remove special characters and accents
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')
    # Remove remaining special chars (keep only letters, numbers, spaces)
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

In [ ]:
# Normalize all country/city columns upfront
traffic['Location_normalized'] = traffic['Location'].apply(normalize_text)
forest['Country_normalized'] = forest['Country Name'].apply(normalize_text)
air_city['Country_normalized'] = air_city['Country'].apply(normalize_text)
air_country['Region_normalized'] = air_country['Region'].apply(normalize_text)
weather['country_normalized'] = weather['country'].apply(normalize_text)


countries_traffic = traffic['Location_normalized'].dropna().unique()
countries_forest = forest['Country_normalized'].dropna().unique()
countries_air_city = air_city['Country_normalized'].dropna().unique()
countries_air_country = air_country['Region_normalized'].dropna().unique()
countries_air_weather = weather['country_normalized'].dropna().unique()

In [28]:
threshold = 0.90

In [ ]:
# Cosine similarity matching function for traffic_countries & forest_countries
def cosine_match(vectorizer, a, b):
    # Transform strings to vectors
    vectors = vectorizer.transform([a, b])
    similarity_score = cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
    return similarity_score >= threshold, similarity_score

# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer().fit(list(countries_traffic)+list(countries_forest))

# Matching process
matches_tf = []
for loc in countries_traffic:
    for country in countries_forest:
        is_match, similarity = cosine_match(vectorizer, loc, country)
        if is_match:
            matches_tf.append({
                "Source": "Traffic",
                "Target": "Forest",
                "Value_A": loc,
                "Value_B": country,
                "Similarity": similarity
            })

# Convert matches to DataFrame
tf_df = pd.DataFrame(matches_tf)

# Display results
if not tf_df.empty:
    print("Traffic-Forest matches (sorted by similarity):")
    print(tf_df.sort_values('Similarity', ascending=False))
else:
    print("No Traffic-Forest matches found")

Traffic-Forest matches (sorted by similarity):
      Source  Target                Value_A                Value_B  Similarity
95   Traffic  Forest  sao tome and principe  sao tome and principe         1.0
0    Traffic  Forest             san marino             san marino         1.0
60   Traffic  Forest              sri lanka              sri lanka         1.0
109  Traffic  Forest           burkina faso           burkina faso         1.0
125  Traffic  Forest           sierra leone           sierra leone         1.0
..       ...     ...                    ...                    ...         ...
51   Traffic  Forest                 brazil                 brazil         1.0
52   Traffic  Forest                 mexico                 mexico         1.0
53   Traffic  Forest                georgia                georgia         1.0
54   Traffic  Forest                 israel                 israel         1.0
150  Traffic  Forest                somalia                somalia         1.0

[151

In [15]:
# Cosine similarity matching function for # traffic_countries & countries_air_country
def cosine_match(vectorizer, a, b):
    # Transform strings to vectors
    vectors = vectorizer.transform([a, b])
    similarity_score = cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
    return similarity_score >= threshold, similarity_score

# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer().fit(list(countries_traffic)+list(countries_air_country))

# Matching process
matches_tf = []
for loc in countries_traffic:
    for country in countries_air_country:
        is_match, similarity = cosine_match(vectorizer, loc, country)
        if is_match:
            matches_tf.append({
                "Source": "Traffic",
                "Target": "Air_Country",
                "Value_A": loc,
                "Value_B": country,
                "Similarity": similarity
            })

# Convert matches to DataFrame
tf_df = pd.DataFrame(matches_tf)

# Display results
if not tf_df.empty:
    print("Traffic-Air_Country matches (sorted by similarity):")
    print(tf_df.sort_values('Similarity', ascending=False))
else:
    print("No Traffic-Forest matches found")

Traffic-Air_Country matches (sorted by similarity):
      Source       Target           Value_A           Value_B  Similarity
0    Traffic  Air_Country        san marino        san marino         1.0
22   Traffic  Air_Country    czech republic    czech republic         1.0
162  Traffic  Air_Country  papua new guinea  papua new guinea         1.0
143  Traffic  Air_Country      sierra leone      sierra leone         1.0
124  Traffic  Air_Country      burkina faso      burkina faso         1.0
..       ...          ...               ...               ...         ...
69   Traffic  Air_Country            russia            russia         1.0
70   Traffic  Air_Country            serbia            serbia         1.0
71   Traffic  Air_Country            turkey            turkey         1.0
72   Traffic  Air_Country          mongolia          mongolia         1.0
85   Traffic  Air_Country            kosovo            kosovo         1.0

[170 rows x 5 columns]


In [20]:
# Cosine similarity matching function for # traffic_countries & countries_air_city
def cosine_match(vectorizer, a, b):
    # Transform strings to vectors
    vectors = vectorizer.transform([a, b])
    similarity_score = cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
    return similarity_score >= threshold, similarity_score

# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer().fit(list(countries_traffic)+list(countries_air_city))

# Matching process
matches_tf = []
for loc in countries_traffic:
    for country in countries_air_city:
        is_match, similarity = cosine_match(vectorizer, loc, country)
        if is_match:
            matches_tf.append({
                "Source": "Traffic",
                "Target": "Air_City",
                "Value_A": loc,
                "Value_B": country,
                "Similarity": similarity
            })

# Convert matches to DataFrame
tf_df = pd.DataFrame(matches_tf)

# Display results
if not tf_df.empty:
    print("Traffic-Air_City matches (sorted by similarity):")
    print(tf_df.sort_values('Similarity', ascending=False))
else:
    print("No Traffic-Forest matches found")

Traffic-Air_City matches (sorted by similarity):
     Source    Target         Value_A         Value_B  Similarity
57  Traffic  Air_City    saudi arabia    saudi arabia         1.0
63  Traffic  Air_City     el salvador     el salvador         1.0
34  Traffic  Air_City      costa rica      costa rica         1.0
0   Traffic  Air_City         andorra         andorra         1.0
51  Traffic  Air_City         albania         albania         1.0
..      ...       ...             ...             ...         ...
25  Traffic  Air_City         austria         austria         1.0
24  Traffic  Air_City        slovenia        slovenia         1.0
23  Traffic  Air_City         belgium         belgium         1.0
22  Traffic  Air_City        slovakia        slovakia         1.0
20  Traffic  Air_City  united kingdom  united kingdom         1.0

[80 rows x 5 columns]


In [21]:
# Cosine similarity matching function for # traffic_countries & countries_air_weather
def cosine_match(vectorizer, a, b):
    # Transform strings to vectors
    vectors = vectorizer.transform([a, b])
    similarity_score = cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
    return similarity_score >= threshold, similarity_score

# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer().fit(list(countries_traffic)+list(countries_air_weather))

# Matching process
matches_tf = []
for loc in countries_traffic:
    for country in countries_air_weather:
        is_match, similarity = cosine_match(vectorizer, loc, country)
        if is_match:
            matches_tf.append({
                "Source": "Traffic",
                "Target": "Air_Weather",
                "Value_A": loc,
                "Value_B": country,
                "Similarity": similarity
            })

# Convert matches to DataFrame
tf_df = pd.DataFrame(matches_tf)

# Display results
if not tf_df.empty:
    print("Traffic-Air_Weather matches (sorted by similarity):")
    print(tf_df.sort_values('Similarity', ascending=False))
else:
    print("No Traffic-Forest matches found")

Traffic-Air_Weather matches (sorted by similarity):
      Source       Target       Value_A       Value_B  Similarity
0    Traffic  Air_Weather    san marino    san marino         1.0
130  Traffic  Air_Weather  sierra leone  sierra leone         1.0
83   Traffic  Air_Weather  south africa  south africa         1.0
84   Traffic  Air_Weather   saint lucia   saint lucia         1.0
97   Traffic  Air_Weather   el salvador   el salvador         1.0
..       ...          ...           ...           ...         ...
55   Traffic  Air_Weather        mexico        mexico         1.0
56   Traffic  Air_Weather       georgia       georgia         1.0
57   Traffic  Air_Weather        israel        israel         1.0
58   Traffic  Air_Weather      barbados      barbados         1.0
155  Traffic  Air_Weather   north korea   north korea         1.0

[156 rows x 5 columns]


In [ ]:
# Matching the cities for each of the relevant tables
# Normalize all city columns upfront
forest['City_normalized'] = forest['Capital'].apply(normalize_text)
air_city['City_normalized'] = air_city['City/Town'].apply(normalize_text)
weather['City_normalized'] = weather['location_name'].apply(normalize_text)


city_forest = forest['City_normalized'].dropna().unique()
city_air_city = air_city['City_normalized'].dropna().unique()
city_air_weather = weather['City_normalized'].dropna().unique()

In [62]:
# Cosine similarity matching function for # city_forest & city_air_city
def cosine_match(vectorizer, a, b):
    # Transform strings to vectors
    vectors = vectorizer.transform([a, b])
    similarity_score = cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
    return similarity_score >= threshold, similarity_score

# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer().fit(list(city_forest)+list(city_air_city))

# Matching process
matches_tf = []
for loc in city_forest:
    for city in city_air_city:
        is_match, similarity = cosine_match(vectorizer, loc, city)
        if is_match:
            matches_tf.append({
                "Source": "City_Forest",
                "Target": "City_Air_City",
                "Value_A": loc,
                "Value_B": city,
                "Similarity": similarity
            })

# Convert matches to DataFrame
tf_df = pd.DataFrame(matches_tf)

# Display results
if not tf_df.empty:
    print("City_Forest - City_Air_City matches (sorted by similarity):")
    print(tf_df.sort_values('Similarity', ascending=False))
else:
    print("No City_Air_City matches found")

City_Forest - City_Air_City matches (sorted by similarity):
         Source         Target       Value_A       Value_B  Similarity
42  City_Forest  City_Air_City   mexico city   mexico city         1.0
56  City_Forest  City_Air_City  san salvador  san salvador         1.0
11  City_Forest  City_Air_City      san jose      san jose         1.0
26  City_Forest  City_Air_City     hong kong     hong kong         1.0
47  City_Forest  City_Air_City     kathmandu     kathmandu         1.0
..          ...            ...           ...           ...         ...
28  City_Forest  City_Air_City      budapest      budapest         1.0
29  City_Forest  City_Air_City       jakarta       jakarta         1.0
30  City_Forest  City_Air_City       douglas       douglas         1.0
31  City_Forest  City_Air_City        tehran        tehran         1.0
65  City_Forest  City_Air_City    montevideo    montevideo         1.0

[66 rows x 5 columns]


In [61]:
# Cosine similarity matching function for # city_forest & city_air_weather
def cosine_match(vectorizer, a, b):
    # Transform strings to vectors
    vectors = vectorizer.transform([a, b])
    similarity_score = cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
    return similarity_score >= threshold, similarity_score

# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer().fit(list(city_forest)+list(city_air_weather))

# Matching process
matches_tf = []
for loc in city_forest:
    for city in city_air_weather:
        is_match, similarity = cosine_match(vectorizer, loc, city)
        if is_match:
            matches_tf.append({
                "Source": "City_Forest",
                "Target": "City_Air_Weather",
                "Value_A": loc,
                "Value_B": city,
                "Similarity": similarity
            })

# Convert matches to DataFrame
tf_df = pd.DataFrame(matches_tf)

# Display results
if not tf_df.empty:
    print("City_Forest - City_Air_Weather matches (sorted by similarity):")
    print(tf_df.sort_values('Similarity', ascending=False))
else:
    print("No City_Air_Weather matches found")

City_Forest - City_Air_Weather matches (sorted by similarity):
          Source            Target         Value_A         Value_B  Similarity
45   City_Forest  City_Air_Weather   santo domingo   santo domingo         1.0
52   City_Forest  City_Air_Weather     addis ababa     addis ababa         1.0
74   City_Forest  City_Air_Weather       new delhi       new delhi         1.0
101  City_Forest  City_Air_Weather     mexico city     mexico city         1.0
67   City_Forest  City_Air_Weather  guatemala city  guatemala city         1.0
..           ...               ...             ...             ...         ...
60   City_Forest  City_Air_Weather           accra           accra         1.0
61   City_Forest  City_Air_Weather         conakry         conakry         1.0
62   City_Forest  City_Air_Weather          banjul          banjul         1.0
63   City_Forest  City_Air_Weather          bissau          bissau         1.0
173  City_Forest  City_Air_Weather          harare          harare  